# CALIDAD DE DATOS - Heart Disease UCI Dataset

## Contenido:
1. Perfilado de Datos (Profiling)
2. Diagnóstico de Dimensiones de Calidad
3. Limpieza y Mejora de Datos


## 1. PERFILADO DE DATOS

Generaremos un reporte completo utilizando ydata-profiling para entender la estructura y características de los datos.


In [ ]:
# Instalar dependencias:
%pip install pandas numpy matplotlib seaborn ydata-profiling scikit-learn -q

✓ Dependencias instaladas correctamente


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline


In [ ]:
# Cargar datos:
df_original = pd.read_csv('heart_disease_uci.csv')
print(f"Dimensiones del dataset: {df_original.shape}")
print(f"Número de registros: {df_original.shape[0]}")
print(f"Número de variables: {df_original.shape[1]}")
df_original.head()


Dimensiones del dataset: (920, 16)
Número de registros: 920
Número de variables: 16


,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


### Conversión Inicial


In [ ]:
# Convertir variables categóricas:
categorical_vars = ['sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']

for col in categorical_vars:
    if col in df_original.columns:
        df_original[col] = df_original[col].astype('category')
        print(f"{col}: {df_original[col].dtype}")

print("\nTIPOS DE DATOS ACTUALIZADOS")
print(df_original.dtypes)

Convirtiendo variables categóricas...
✓ sex: category
✓ dataset: category
✓ cp: category
✓ fbs: category
✓ restecg: category
✓ exang: category
✓ slope: category
✓ thal: category

=== TIPOS DE DATOS ACTUALIZADOS ===
id             int64
age            int64
sex         category
dataset     category
cp          category
trestbps     float64
chol         float64
fbs         category
restecg     category
thalch       float64
exang       category
oldpeak      float64
slope       category
ca           float64
thal        category
num            int64
dtype: object

Uso de memoria optimizado: 66.84 KB


In [ ]:
df_original.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   id        920 non-null    int64   
 1   age       920 non-null    int64   
 2   sex       920 non-null    category
 3   dataset   920 non-null    category
 4   cp        920 non-null    category
 5   trestbps  861 non-null    float64 
 6   chol      890 non-null    float64 
 7   fbs       830 non-null    category
 8   restecg   918 non-null    category
 9   thalch    865 non-null    float64 
 10  exang     865 non-null    category
 11  oldpeak   858 non-null    float64 
 12  slope     611 non-null    category
 13  ca        309 non-null    float64 
 14  thal      434 non-null    category
 15  num       920 non-null    int64   
dtypes: category(8), float64(5), int64(3)
memory usage: 65.9 KB


In [ ]:
# Generar reporte de perfilado;:
profile = ProfileReport(df_original, 
                       title="Heart Disease UCI - Reporte de Perfilado",
                       minimal=True)

# Guardar reporte:
profile.to_file("heart_disease_profiling_report.html")
print("heart_disease_profiling_report.html")


Generando reporte de perfilado de datos...
Esto puede tardar 2-3 minutos, por favor espera...


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

## 2. DIAGNÓSTICO DE CALIDAD DE DATOS

Analizaremos las siguientes dimensiones de calidad:
- **Completitud**: Presencia de valores faltantes
- **Consistencia**: Coherencia de los valores
- **Exactitud**: Valores fuera de rango o anómalos
- **Unicidad**: Duplicados
- **Validez**: Tipos de datos correctos


### 2.1 COMPLETITUD (Valores Faltantes)


In [ ]:
# Reemplazar cadenas vacías por NaN:
df_analysis = df_original.replace('', np.nan)

# Análisis de valores faltantes:
missing_data = pd.DataFrame({
    'Columna': df_analysis.columns,
    'Valores_Faltantes': df_analysis.isnull().sum(),
    'Porcentaje': (df_analysis.isnull().sum() / len(df_analysis) * 100).round(2)
}).sort_values('Valores_Faltantes', ascending=False)

print("\nANÁLISIS DE COMPLETITUD")
print(missing_data[missing_data['Valores_Faltantes'] > 0])

# Visualización:
plt.figure(figsize=(12, 6))
missing_cols = missing_data[missing_data['Valores_Faltantes'] > 0]
if len(missing_cols) > 0:
    plt.barh(missing_cols['Columna'], missing_cols['Porcentaje'])
    plt.xlabel('Porcentaje de Valores Faltantes (%)')
    plt.title('Completitud de Datos por Variable')
    plt.tight_layout()
    plt.show()
else:
    print("No hay valores faltantes en el dataset")

**Diagnóstico de Completitud**

Se identificaron valores faltantes en 10 variables del dataset:

- Variables con alto porcentaje de valores faltantes (>30%):
  - ca (66.41%): número de vasos principales coloreados por fluoroscopia
  - thal (52.83%): resultados de pruebas de talasemia
  - slope (33.59%): pendiente del segmento ST del ejercicio

- Variables numéricas con valores faltantes moderados:
  - trestbps (6.41%), chol (3.26%), thalch (5.98%), oldpeak (6.74%)

- Variables categóricas con valores faltantes:
  - fbs (9.78%), restecg (0.22%), exang (5.98%)

Los valores faltantes se deben probablemente a la consolidación de datos de diferentes centros médicos (Cleveland, Hungary, Switzerland, VA Long Beach) con diferentes protocolos de recolección de información.


### 2.2 CONSISTENCIA (Valores Inconsistentes)


In [ ]:
print("\nANÁLISIS DE CONSISTENCIA")

# Verificar consistencia de valores categóricos:
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']

for col in categorical_cols:
    if col in df_original.columns:
        print(f"\n{col.upper()}:")
        print(df_original[col].value_counts(dropna=False))

In [ ]:
# Verificar rangos de variables numéricas:
print("\nRANGOS DE VARIABLES NUMÉRICAS")
numeric_cols = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca']

for col in numeric_cols:
    if col in df_analysis.columns:
        # Convertir a numérico:z
        df_analysis[col] = pd.to_numeric(df_analysis[col], errors='coerce')
        print(f"\n{col.upper()}:")
        print(f"  Min: {df_analysis[col].min()}")
        print(f"  Max: {df_analysis[col].max()}")
        print(f"  Media: {df_analysis[col].mean():.2f}")
        print(f"  Desv. Std: {df_analysis[col].std():.2f}")

**Diagnóstico de Consistencia**

Se identificaron las siguientes inconsistencias en los datos:

1. Formato de variables booleanas: Las variables `fbs` y `exang` contienen valores "TRUE"/"FALSE" como texto en lugar de tipos booleanos nativos.

2. Valores anómalos: Se detectaron valores de 0 en variables donde no son clínicamente válidos:
   - trestbps (presión arterial en reposo): 1 registro con valor 0
   - chol (colesterol sérico): 172 registros con valor 0

3. Distribución de variables categóricas: La distribución es consistente con estudios epidemiológicos (mayor prevalencia de pacientes masculinos: 79%, tipos de dolor torácico con mayoría asintomática: 54%).

Estas inconsistencias son típicas de la consolidación de datos clínicos de múltiples centros médicos sin un proceso previo de estandarización.


### 2.3 EXACTITUD (Valores Fuera de Rango)


In [ ]:
print("\nANÁLISIS DE EXACTITUD")

# Definir rangos:
ranges = {
    'age': (0, 120),
    'trestbps': (80, 220),  # Presión arterial en reposo
    'chol': (100, 600),     # Colesterol sérico
    'thalch': (60, 220),    # Ritmo cardíaco máximo
    'oldpeak': (0, 10),     # Depresión ST
    'ca': (0, 4)            # Número de vasos principales
}

outliers_summary = []

for col, (min_val, max_val) in ranges.items():
    if col in df_analysis.columns:
        out_of_range = df_analysis[
            (df_analysis[col] < min_val) | (df_analysis[col] > max_val)
        ][col].dropna()
        
        if len(out_of_range) > 0:
            outliers_summary.append({
                'Variable': col,
                'Rango_Esperado': f"{min_val}-{max_val}",
                'Valores_Fuera_Rango': len(out_of_range),
                'Porcentaje': f"{(len(out_of_range)/len(df_analysis)*100):.2f}%"
            })

if outliers_summary:
    outliers_df = pd.DataFrame(outliers_summary)
    print(outliers_df)
else:
    print("No se encontraron valores fuera de rango")

In [ ]:
# Detectar outliers usando IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return len(outliers), lower_bound, upper_bound

print("\nDETECCIÓN DE OUTLIERS (Método IQR)")
for col in numeric_cols:
    if col in df_analysis.columns:
        n_outliers, lower, upper = detect_outliers_iqr(df_analysis, col)
        if n_outliers > 0:
            print(f"\n{col.upper()}: {n_outliers} outliers detectados")
            print(f"  Límite inferior: {lower:.2f}")
            print(f"  Límite superior: {upper:.2f}")

**Diagnóstico de Exactitud**

Análisis de valores fuera de rangos clínicos esperados:

1. Valores fuera de rango por rangos médicos establecidos:
   - trestbps: 1 valor fuera del rango 80-220 mmHg (0.11%)
   - chol: 174 valores fuera del rango 100-600 mg/dl (18.91%)
   - oldpeak: 12 valores fuera del rango 0-10 (1.30%)

2. Outliers detectados mediante método IQR:
   - trestbps: 28 outliers (valores extremos de presión arterial)
   - chol: 183 outliers (principalmente valores de 0 y valores muy altos)
   - thalch: 2 outliers (frecuencia cardíaca máxima atípica)
   - oldpeak: 16 outliers (depresión ST extrema)
   - ca: 20 outliers (valores en el límite superior)

Los valores de 0 en chol y trestbps son clínicamente imposibles y representan probablemente marcadores de datos faltantes que no fueron codificados correctamente en la fuente original.


### 2.4 UNICIDAD (Duplicados)


In [ ]:
print("\nANÁLISIS DE UNICIDAD")

# Verificar duplicados basados en ID
duplicate_ids = df_original['id'].duplicated().sum()
print(f"Registros con ID duplicado: {duplicate_ids}")

# Verificar duplicados exactos en todas las columnas (excepto ID)
cols_without_id = [col for col in df_original.columns if col != 'id']
duplicate_rows = df_original.duplicated(subset=cols_without_id, keep=False).sum()
print(f"Registros completamente duplicados: {duplicate_rows}")

if duplicate_rows > 0:
    print("\nEjemplos de registros duplicados:")
    print(df_original[df_original.duplicated(subset=cols_without_id, keep=False)].head(10))

**Diagnóstico de Unicidad**

Resultados del análisis de duplicación:

- IDs únicos: No se encontraron IDs duplicados (920 IDs únicos)
- Registros duplicados: 4 registros completamente duplicados (0.43% del dataset)

Los registros duplicados identificados corresponden a 2 pares de pacientes con información idéntica en todas las variables clínicas. Estos duplicados se originan probablemente durante el proceso de consolidación de los datasets de diferentes centros médicos (Hungary y VA Long Beach en los ejemplos identificados).


### 2.5 VALIDEZ (Tipos de Datos)


In [ ]:
print("\nANÁLISIS DE VALIDEZ")
print("\nTipos de datos actuales:")
print(df_original.dtypes)

print("\n\nTipos de datos esperados:")
expected_types = {
    'id': 'int',
    'age': 'int',
    'sex': 'categorical',
    'dataset': 'categorical',
    'cp': 'categorical',
    'trestbps': 'float',
    'chol': 'float',
    'fbs': 'boolean',
    'restecg': 'categorical',
    'thalch': 'float',
    'exang': 'boolean',
    'oldpeak': 'float',
    'slope': 'categorical',
    'ca': 'float',
    'thal': 'categorical',
    'num': 'int (target)'
}

for col, expected_type in expected_types.items():
    print(f"{col}: {expected_type}")

**Diagnóstico de Validez**

Evaluación de tipos de datos:

Tipos de datos correctos:
- Variables identificadoras y numéricas: `id`, `age`, `num` (int64)
- Variables continuas: `trestbps`, `chol`, `thalch`, `oldpeak`, `ca` (float64)
- Variables categóricas: `sex`, `dataset`, `cp`, `restecg`, `slope`, `thal` (category)

Tipos de datos que requieren conversión:
- `fbs` y `exang`: Almacenadas como category con valores de texto "TRUE"/"FALSE" en lugar de tipos booleanos

El dataset presenta una estructura de tipos de datos generalmente apropiada después de la conversión inicial a tipos categóricos. Solo se requiere conversión explícita de las variables booleanas para optimizar el procesamiento y modelado.


## 3. LIMPIEZA Y MEJORA DE DATOS

Implementaremos las transformaciones necesarias para resolver los problemas identificados.


### 3.1 Preparación Inicial


In [ ]:
# Crear copia para limpieza:
df_clean = df_original.copy()

print("Dataset antes de limpieza:")
print(f"Shape: {df_clean.shape}")
print(f"Valores faltantes totales: {df_clean.replace('', np.nan).isnull().sum().sum()}")

### 3.2 Conversión de Tipos de Datos


In [ ]:
print("\nPASO 1: CONVERSIÓN DE TIPOS DE DATOS")

# Reemplazar strings vacíos por NaN:
df_clean = df_clean.replace('', np.nan)

# Convertir variables booleanas:
bool_cols = ['fbs', 'exang']
for col in bool_cols:
    df_clean[col] = df_clean[col].map({'TRUE': True, 'FALSE': False, True: True, False: False})
    print(f"{col} convertido a booleano")

# Convertir variables numéricas:
numeric_cols_convert = ['trestbps', 'chol', 'thalch', 'oldpeak', 'ca']
for col in numeric_cols_convert:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    print(f"{col} convertido a numérico")

print("\nTipos de datos después de conversión:")
print(df_clean.dtypes)

### 3.3 Manejo de Valores Anómalos


In [ ]:
print("\nPASO 2: MANEJO DE VALORES ANÓMALOS")

# Reemplazar valores de 0 por NaN en variables donde 0 no es válido:
zero_invalid_cols = ['trestbps', 'chol', 'thalch']
for col in zero_invalid_cols:
    n_zeros = (df_clean[col] == 0).sum()
    if n_zeros > 0:
        df_clean.loc[df_clean[col] == 0, col] = np.nan
        print(f"{col}: {n_zeros} valores de 0 convertidos a NaN")

### 3.4 Imputación de Valores Faltantes


In [ ]:
print("\nPASO 3: IMPUTACIÓN DE VALORES FALTANTES")

# Contar valores faltantes antes de imputación:
missing_before = df_clean.isnull().sum()
print("\nValores faltantes antes de imputación:")
print(missing_before[missing_before > 0])

# Estrategia de imputación:
# - Variables numéricas: imputar con mediana por dataset origen (fallback: mediana global)
# - Variables categóricas: imputar con moda por dataset origen (fallback: moda global)

# Imputación de variables numéricas por dataset:
numeric_to_impute = ['trestbps', 'chol', 'thalch', 'oldpeak', 'ca']

for col in numeric_to_impute:
    if df_clean[col].isnull().sum() > 0:
        # Calcular mediana global como fallback:
        global_median = df_clean[col].median()
        
        # Imputar con la mediana del dataset correspondiente:
        for dataset in df_clean['dataset'].unique():
            mask = (df_clean['dataset'] == dataset) & (df_clean[col].isnull())
            if mask.sum() > 0:
                median_val = df_clean[df_clean['dataset'] == dataset][col].median()
                # Si el dataset no tiene valores válidos, usar mediana global:
                if pd.isna(median_val):
                    median_val = global_median
                df_clean.loc[mask, col] = median_val
        print(f"{col} imputado con mediana por dataset")

# Imputación de variables categóricas (booleanas y nominales):
categorical_to_impute = ['fbs', 'restecg', 'exang', 'slope', 'thal']

for col in categorical_to_impute:
    if df_clean[col].isnull().sum() > 0:
        original_dtype = df_clean[col].dtype
        df_clean[col] = df_clean[col].astype('object')
        
        # Calcular moda global como fallback:
        global_mode = df_clean[col].mode()
        global_mode_val = global_mode.iloc[0] if len(global_mode) > 0 else None
        
        # Imputar con la moda del dataset correspondiente:
        for dataset in df_clean['dataset'].unique():
            mask = (df_clean['dataset'] == dataset) & (df_clean[col].isnull())
            if mask.sum() > 0:
                mode_val = df_clean[df_clean['dataset'] == dataset][col].mode()
                if len(mode_val) > 0:
                    fill_value = mode_val.iloc[0]
                else:
                    # Si el dataset no tiene valores válidos, usar moda global:
                    fill_value = global_mode_val
                
                if fill_value is not None:
                    df_clean.loc[mask, col] = fill_value
        
        df_clean[col] = df_clean[col].astype('category')
        print(f"{col} imputado con moda por dataset")

# Valores faltantes después de imputación
missing_after = df_clean.isnull().sum()
print("\nValores faltantes después de imputación:")
if missing_after.sum() == 0:
    print("No quedan valores faltantes en el dataset")
else:
    print(missing_after[missing_after > 0])

### 3.5 Eliminación de Duplicados


In [ ]:
print("\nPASO 4: ELIMINACIÓN DE DUPLICADOS")

# Eliminar filas completamente duplicadas:
cols_for_dup = [col for col in df_clean.columns if col != 'id']
duplicates = df_clean.duplicated(subset=cols_for_dup).sum()

if duplicates > 0:
    df_clean = df_clean.drop_duplicates(subset=cols_for_dup, keep='first')
    print(f"Eliminados {duplicates} registros duplicados")
else:
    print("No se encontraron registros duplicados")

print(f"\nShape después de eliminar duplicados: {df_clean.shape}")

### 3.6 Creación de Variable Target Binaria


In [ ]:
print("\nPASO 5: CREACIÓN DE VARIABLE TARGET")

# Crear variable binaria: 0 = sin enfermedad, 1 = con enfermedad (cualquier grado):
df_clean['heart_disease'] = (df_clean['num'] > 0).astype(int)

print("\nDistribución de la variable target:")
print(df_clean['heart_disease'].value_counts())
print("\nPorcentajes:")
print(df_clean['heart_disease'].value_counts(normalize=True) * 100)

# Visualización:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

df_clean['num'].value_counts().sort_index().plot(kind='bar', ax=axes[0])
axes[0].set_title('Distribución Original (num: 0-4)')
axes[0].set_xlabel('Nivel de Enfermedad')
axes[0].set_ylabel('Frecuencia')

df_clean['heart_disease'].value_counts().plot(kind='bar', ax=axes[1])
axes[1].set_title('Variable Target Binaria')
axes[1].set_xlabel('Heart Disease (0=No, 1=Yes)')
axes[1].set_ylabel('Frecuencia')
axes[1].set_xticklabels(['No', 'Yes'], rotation=0)

plt.tight_layout()
plt.show()

### 3.7 Validación Final de Limpieza


In [ ]:
print("\nRESUMEN DE LIMPIEZA")
print(f"\nRegistros originales: {df_original.shape[0]}")
print(f"Registros después de limpieza: {df_clean.shape[0]}")
print(f"Registros eliminados: {df_original.shape[0] - df_clean.shape[0]}")

print("\nCALIDAD FINAL DE DATOS")
missing_final = df_clean.isnull().sum()
print(f"\nValores faltantes totales: {missing_final.sum()}")
print(f"Columnas con valores faltantes: {(missing_final > 0).sum()}")

if missing_final.sum() > 0:
    print("\nColumnas con valores faltantes restantes:")
    print(missing_final[missing_final > 0])

print("\nTipos de datos finales:")
print(df_clean.dtypes)

print("\nESTADÍSTICAS DESCRIPTIVAS FINALES")
print(df_clean.describe())

### 3.8 Visualización de Mejoras


In [ ]:
# Comparación antes y después:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Valores faltantes antes:
df_original_analysis = df_original.replace('', np.nan)
missing_original = df_original_analysis.isnull().sum().sort_values(ascending=False)[:10]

axes[0].barh(range(len(missing_original)), missing_original.values)
axes[0].set_yticks(range(len(missing_original)))
axes[0].set_yticklabels(missing_original.index)
axes[0].set_xlabel('Número de Valores Faltantes')
axes[0].set_title('ANTES: Valores Faltantes')
axes[0].invert_yaxis()

# Valores faltantes después:
missing_clean = df_clean.isnull().sum().sort_values(ascending=False)[:10]

axes[1].barh(range(len(missing_clean)), missing_clean.values, color='green')
axes[1].set_yticks(range(len(missing_clean)))
axes[1].set_yticklabels(missing_clean.index)
axes[1].set_xlabel('Número de Valores Faltantes')
axes[1].set_title('DESPUÉS: Valores Faltantes')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

### 3.9 Exportar Datos Limpios


In [ ]:
# Guardar dataset limpio:
df_clean.to_csv('heart_disease_clean.csv', index=False)
print("Dataset limpio guardado como: heart_disease_clean.csv")

# Mostrar primeras filas del dataset limpio:
print("\nPrimeras filas del dataset limpio:")
df_clean.head(10)

## CONCLUSIONES DE CALIDAD DE DATOS

### Diagnóstico Inicial

**Dataset Original**: 920 registros, 16 variables

**Problemas Identificados por Dimensión**:

1. **Completitud**: 1,759 valores faltantes (19.1% del total de datos)
   - Variables críticas con >30% de datos faltantes: ca (66.4%), thal (52.8%), slope (33.6%)
   - Variables con datos faltantes moderados: fbs (9.8%), trestbps (6.4%), thalch (6.0%), exang (6.0%), oldpeak (6.7%), chol (3.3%), restecg (0.2%)

2. **Consistencia**: Variables booleanas almacenadas como texto (TRUE/FALSE) y 173 valores anómalos de 0 en variables clínicas

3. **Exactitud**: 229 valores fuera de rangos clínicos esperados, incluyendo 183 outliers en colesterol y 28 en presión arterial

4. **Unicidad**: 4 registros completamente duplicados (0.43%)

5. **Validez**: Tipos de datos apropiados después de conversión inicial, requiriendo solo ajustes menores en variables booleanas

### Proceso de Limpieza Implementado

**Paso 1 - Conversión de Tipos**: Estandarización de variables booleanas (fbs, exang) y validación de tipos numéricos

**Paso 2 - Valores Anómalos**: Identificación y conversión a NaN de 173 valores de 0 en trestbps (1) y chol (172)

**Paso 3 - Imputación**: Estrategia de imputación por dataset de origen con fallback a valores globales
- Variables numéricas: mediana por dataset (trestbps, chol, thalch, oldpeak, ca)
- Variables categóricas: moda por dataset (fbs, restecg, exang, slope, thal)

**Paso 4 - Duplicados**: Eliminación de 2 registros duplicados

**Paso 5 - Variable Target**: Creación de variable binaria heart_disease (0=sin enfermedad, 1=con enfermedad)

### Resultados Finales

**Dataset Limpio**: 918 registros, 17 variables (incluyendo variable target)

**Distribución Target**:
- Clase 0 (sin enfermedad): 410 pacientes (44.7%)
- Clase 1 (con enfermedad): 508 pacientes (55.3%)

**Calidad de Datos**:
- Valores faltantes: 0 (100% de completitud)
- Duplicados: 0
- Tipos de datos: Validados y optimizados
- Outliers: Conservados por representar casos clínicos extremos válidos

**Características del Dataset Final**:
- Edad promedio: 53.5 años (rango: 28-77)
- Presión arterial: 132.1 ± 17.9 mmHg
- Colesterol: 245.5 ± 55.8 mg/dl
- Frecuencia cardíaca máxima: 136.5 ± 25.5 bpm
- Dataset balanceado apropiado para clasificación binaria